# Deriving a Modern, Pythonic API for Anki files

Anki's native file format describe a _Collection_ of _Decks_ of _Cards_ (made of notes). On disk, an `.apkg` is a zip archive containing: 
- media resources 
- a `sqlite3` database

While written in python, `anki` carries too many dependencies to be used effectively by downstreams.

In [ ]:
from pathlib import Path
import sqlalchemy as SA
import alchemyjsonschema as AJ
import zipfile
import tempfile
import shutil
from collections import defaultdict
import IPython
import isort
import black
import libcst
import re

## Goal

Our goal is a ready-to-package python module with
- well-typed classes
- JSON serialization
- documentation
- tests
- async-compatible

In [ ]:
MOD = Path("_anki_schema")
VERSION = "2021.03.0"
shutil.rmtree(MOD, ignore_errors=True)
MOD.mkdir()
(MOD / "__init__.py").write_text(f'''
"""a modern, pythonic API for Anki files
"""
__version__ = "{VERSION}"
''')

We'll eventually want to look at a bunch of `.apkg` files.

In [ ]:
APKG = [*Path("vendor/ankitects/anki").rglob("*.apkg")]
print(APKG)

But first, let's just look at one.

In [ ]:
apkg = zipfile.ZipFile(APKG[0])
apkg.infolist()

First, we can use [alchemyjsonschema](https://pypi.org/project/alchemyjsonschema) to derive [SQLAlchemy 1.3](https://docs.sqlalchemy.org/en/13) classes.

In [ ]:
with tempfile.TemporaryDirectory() as td:
    tdp = Path(td)
    apkg.extractall(tdp)
    conn_str = f"""sqlite:///{tdp / "collection.anki2"}"""
    raw_sa_py = !sqlacodegen {conn_str}

Helpfully, the nice folks at [AnkiDroid](https://github.com/ankidroid/Anki-Android) have annotated the database schema.

In [ ]:
annotated_db_schema = Path("vendor/ankidroid/Anki-Android.wiki/Database-Structure.md").read_text().split("```sql")[1].split("```")[0]
tables = dict(re.findall(r"""CREATE TABLE (.*?) \((.*?)\);""", annotated_db_schema, flags=re.M | re.S))
fields = defaultdict(lambda: defaultdict(list))
for table, text in tables.items():
    field = None
    for line in text.splitlines():
        match = re.match(r"^\s*([^\-\s]*)", line)
        if match and match.groups() and match.groups()[0]:
            field = match.groups()[0]
        elif re.match(r'\s+-- ', line) is not None:
            fields[table][field] += [re.sub("^\s+-- ", "", line)]
display(IPython.display.JSON(fields))

From the raw extracted python, we'll create our base model.

In [ ]:
annotated_sa_py = []
clazz = None
nl = "\n"
for old_line in raw_sa_py:
    new_line = old_line
    if "__tablename__" in old_line:
        table = re.findall(r"__tablename__ = '(.*)'", old_line)[0].lower()
        assert table in fields.keys()
        print(table)
    elif "= Column" in old_line:
        field = old_line.split(" = ")[0].strip()
        print("...", field)
        new_line = old_line.replace(")", f""", doc='''{nl.join(fields[table][field]).strip()}''')""")
    annotated_sa_py += [new_line]

In [ ]:
BASE = MOD / "_base.py"
BASE.write_text(black.format_str("\n".join(annotated_sa_py[1:]), mode=black.FileMode(line_length=88)))

Now we can get a first look at the JSON schema.

In [ ]:
SCHEMA = MOD / "schema"
SCHEMA.mkdir(exist_ok=True)
RAW = SCHEMA / "raw.schema.json"
!PYTHONPATH=. alchemyjsonschema --walker structural _anki_schema._base > {RAW}
IPython.display.Markdown(f"### [The Raw Schema]({RAW.as_posix()})")

Now we can use this to build `pydantic` models.

In [ ]:
PYDANTIC = MOD / "_pydantic.py"
PYDANTIC.write_text('''"""pydantic models for anki"""
from pydantic_sqlalchemy import sqlalchemy_to_pydantic
from . import _base
Card = sqlalchemy_to_pydantic(_base.Card)
Col = sqlalchemy_to_pydantic(_base.Col)
Note = sqlalchemy_to_pydantic(_base.Note)
Revlog = sqlalchemy_to_pydantic(_base.Revlog)
''')

Now we should be able to actually import these things.

In [ ]:
from _anki_schema._pydantic import Card, Col, Note, Revlog

In [ ]:
import pytest
import pydantic

and they should be suitably _pydantic_.

In [ ]:
for Clazz in [Card, Col, Note, Revlog]:
    with pytest.raises(pydantic.ValidationError):
        print(f"{Clazz} is pydantic")
        Clazz()
        print("uhoh, shouldn't have gotten here")

Next we'll build some convenience classes around them.

# ... a miracle happens

Finally, we should have a `Collection` which we can conveniently work with.

```python
foo = Collection("foo.anki2") 
with foo.open() as foo:
    card = foo.models.MultipleChoice(
        "the good",
        "the bad",
        "the ugly
    )
```